In [ ]:
!pip install openai python-dotenv gradio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
import os, time, gradio as gr
from openai import OpenAI

load_dotenv(override=True)

c:\Users\tiny\Code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuración de modelos en la nube

In [50]:
from collections import namedtuple
from functools import reduce

Provider = namedtuple('Provider', ['name', 'base_url', 'api_key', 'models'])
Task = namedtuple('Task', ['name', 'prompt'])

providers = [
    Provider(
        'open_router', 'https://openrouter.ai/api/v1', 'OPEN_ROUTER_API_KEY',
        ["gpt-3.5-turbo", "deepseek/deepseek-r1:free", "gpt-4o-mini", "meta-llama/llama-3.1-8b-instruct:free"]
    ),
    Provider(
        'gemini', 'https://generativelanguage.googleapis.com/v1beta/openai/', 'GEMINI_API_KEY',
        ["gemini-2.5-flash", "gemini-2.5-pro"]
    ),
    Provider(
        'groq', 'https://api.groq.com/openai/v1', 'GROQ_API_KEY',
        ["llama-3.3-70b-versatile"]
    ),
]

tasks = [
    Task('translation', 'translate the following text to spanish, provide only the translation: processing_text'),
    Task('summarization', 'summarize the following text to number_of_words words, provide only the summarization: processing_text')
]

model_names = list(reduce(lambda x, y: [*x, *y], map(lambda x: x.models, providers), []))
task_names = list(map(lambda x: x.name, tasks))

In [51]:
import time

def get_prompt(task_name, text, number_of_words=None):
    task = next(filter(lambda x: x.name == task_name, tasks))
    prompt: str = task.prompt
    prompt = prompt.replace('number_of_words', str(number_of_words))
    prompt = prompt.replace('processing_text', text)
    return prompt

def get_provider_api(model):
    provider = next(filter(lambda x: model in x.models, providers))
    api_key = os.getenv(provider.api_key)
    api = OpenAI(api_key=api_key, base_url=provider.base_url)
    return api


def process_text(model, task, text, number_of_words=None):
    if not text.strip():
        return "Provide a text please..."
    
    start_time = time.time()
    prompt = get_prompt(task, text, number_of_words)
    
    try:
        provider_api = get_provider_api(model)
        response = provider_api.chat.completions.create(
            model=model,
            messages=[{'role': 'user', 'content': prompt}],
            max_tokens=number_of_words
        )
    except Exception as e:
        return f"⚠️ Error connecting to {provider_api.base_url}: {str(e)}"

    output = response.choices[0].message.content
    end_time = time.time()
    elapsed = round(end_time - start_time, 2)
    return output, f'{elapsed}s'


### Interfaz con gradio

In [ ]:
css = """
#title {
    text-align: center;
}

.contain {
    display: flex;
    justify-content: center;
    align-items: center;
}
"""

with gr.Blocks(css=css) as demo:
  with gr.Column(elem_id='app'):
    with gr.Column():
        gr.Markdown('# ChatMC', elem_id='title')
        
        with gr.Row():
            with gr.Column():
                input = gr.TextArea(placeholder='Input text...', label='Input')

            with gr.Column():
                output = gr.TextArea(
                    placeholder='Output will appear here...', 
                    label='Output', 
                    show_copy_button=True
                )

        with gr.Row(equal_height=True):
            with gr.Column():
                process_button = gr.Button("Process")
            with gr.Row(equal_height=True):
                elapsed_time = gr.Label(label='Processing time')
                use_as_input = gr.Button('Use as Input')
        
        with gr.Row():
            model = gr.Dropdown(model_names, label="Model")
            task = gr.Dropdown(task_names, label='Task')
            number_of_words = gr.Number(None, label='Max number of words')
    
    use_as_input.click(lambda x: x, inputs=[output], outputs=[input])

    
    process_button.click(process_text, inputs=[model, task, input, number_of_words], outputs=[output, elapsed_time])

demo.launch()

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.
